In [2]:
import pandas as pd
import subprocess
import shutil
import runpy
import json
import glob
import sys
import os
from collections import Counter, defaultdict
from pathlib import Path



DATA_DIR = Path('/media/vanafa/1TB/Workspace/Cuatri 6/ML/TP Final/Data')
assert DATA_DIR.exists(), f'Data directory not found: {DATA_DIR}'
csv_files = sorted([str(p) for p in DATA_DIR.rglob('*.csv')])
print(f'Encontrados {len(csv_files)} archivos CSV')

Encontrados 56 archivos CSV


In [3]:
for f in csv_files:
    df = pd.read_csv(f, nrows=5)
print('Columnas dynamic len:', len(df.columns))

Columnas dynamic len: 7


In [4]:
# archivo dinámico
f = csv_files[0]
df0 = pd.read_csv(f, nrows=1000)
display(df0.head())
print('Dtypes:')
print(df0.dtypes)

,DayNum,VehId,Trip,Timestamp(ms),Latitude[deg],Longitude[deg],Vehicle Speed[km/h],MAF[g/sec],Engine RPM[RPM],Absolute Load[%],...,Air Conditioning Power[kW],Air Conditioning Power[Watts],Heater Power[Watts],HV Battery Current[A],HV Battery SOC[%],HV Battery Voltage[V],Short Term Fuel Trim Bank 1[%],Short Term Fuel Trim Bank 2[%],Long Term Fuel Trim Bank 1[%],Long Term Fuel Trim Bank 2[%]
0,1.586651,8,706,0,42.277558,-83.69875,40.0,22.129999,2285.0,49.019608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.586651,8,706,200,42.277558,-83.69875,40.0,22.129999,2285.0,67.450981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.586651,8,706,1100,42.277558,-83.69875,45.0,22.129999,2285.0,67.450981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.586651,8,706,2100,42.277558,-83.69875,47.0,6.150000,2744.0,67.450981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.586651,8,706,4200,42.277558,-83.69875,48.0,21.440001,1982.0,67.450981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Dtypes:
DayNum                            float64
VehId                               int64
Trip                                int64
Timestamp(ms)                       int64
Latitude[deg]                     float64
Longitude[deg]                    float64
Vehicle Speed[km/h]               float64
MAF[g/sec]                        float64
Engine RPM[RPM]                   float64
Absolute Load[%]                  float64
OAT[DegC]                         float64
Fuel Rate[L/hr]                   float64
Air Conditioning Power[kW]        float64
Air Conditioning Power[Watts]     float64
Heater Power[Watts]               float64
HV Battery Current[A]             float64
HV Battery SOC[%]                 float64
HV Battery Voltage[V]             float64
Short Term Fuel Trim Bank 1[%]    float64
Short Term Fuel Trim Bank 2[%]    float64
Long Term Fuel Trim Bank 1[%]     float64
Long Term Fuel Trim Bank 2[%]     float64
dtype: object


In [5]:
total_rows = 0
for f in csv_files:
    with open(f, 'rb') as fh:
        cnt = 0
        for _ in fh:
            cnt += 1
    r = max(0, cnt - 1)
    total_rows += r
print('Samples:', total_rows)

Samples: 22437192


In [6]:
dir_path = DATA_DIR / 'VED_DynamicData'
files = sorted([p for p in csv_files if str(dir_path) in p])
if not files:
    raise FileNotFoundError(f'No files found in {dir_path}')
first_file = files[0]
print('First file:', first_file)
df_first = pd.read_csv(first_file, nrows=20)
display(df_first.head())

First file: /media/vanafa/1TB/Workspace/Cuatri 6/ML/TP Final/Data/VED_DynamicData/VED_171101_week.csv


,DayNum,VehId,Trip,Timestamp(ms),Latitude[deg],Longitude[deg],Vehicle Speed[km/h],MAF[g/sec],Engine RPM[RPM],Absolute Load[%],...,Air Conditioning Power[kW],Air Conditioning Power[Watts],Heater Power[Watts],HV Battery Current[A],HV Battery SOC[%],HV Battery Voltage[V],Short Term Fuel Trim Bank 1[%],Short Term Fuel Trim Bank 2[%],Long Term Fuel Trim Bank 1[%],Long Term Fuel Trim Bank 2[%]
0,1.586651,8,706,0,42.277558,-83.69875,40,22.129999,2285,49.019608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.586651,8,706,200,42.277558,-83.69875,40,22.129999,2285,67.450981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.586651,8,706,1100,42.277558,-83.69875,45,22.129999,2285,67.450981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.586651,8,706,2100,42.277558,-83.69875,47,6.150000,2744,67.450981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.586651,8,706,4200,42.277558,-83.69875,48,21.440001,1982,67.450981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# for file in dir_path go looking at each column and create a new file with the columns that are not full of nans. save those files in a new folder
output_dir = DATA_DIR / 'VED_DynamicData_cleaned'
output_dir.mkdir(exist_ok=True)

for file_path in files:
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    original_cols = df.shape[1]
    
    # Remove columns where ALL values are NaN (including the header)
    df_cleaned = df.dropna(axis=1, how='all')
    
    cleaned_cols = df_cleaned.shape[1]
    removed_cols = original_cols - cleaned_cols
    
    # Get the filename and save to output directory
    filename = Path(file_path).name
    output_path = output_dir / filename
    df_cleaned.to_csv(output_path, index=False)
    
    print(f'Processed: {filename} | Original: {original_cols} cols | Cleaned: {cleaned_cols} cols | Removed: {removed_cols} cols')

print(f'\nAll files cleaned and saved to: {output_dir}')

Processed: VED_171101_week.csv | Original columns: 22 | Cleaned columns: 22
Processed: VED_171108_week.csv | Original columns: 22 | Cleaned columns: 22
Processed: VED_171108_week.csv | Original columns: 22 | Cleaned columns: 22
Processed: VED_171115_week.csv | Original columns: 22 | Cleaned columns: 22
Processed: VED_171115_week.csv | Original columns: 22 | Cleaned columns: 22
Processed: VED_171122_week.csv | Original columns: 22 | Cleaned columns: 22
Processed: VED_171122_week.csv | Original columns: 22 | Cleaned columns: 22
Processed: VED_171129_week.csv | Original columns: 22 | Cleaned columns: 22
Processed: VED_171129_week.csv | Original columns: 22 | Cleaned columns: 22
Processed: VED_171206_week.csv | Original columns: 22 | Cleaned columns: 22
Processed: VED_171206_week.csv | Original columns: 22 | Cleaned columns: 22
Processed: VED_171213_week.csv | Original columns: 22 | Cleaned columns: 22
Processed: VED_171213_week.csv | Original columns: 22 | Cleaned columns: 22
Processed: V